## Долгосрочная работа по теме алгоритм Берлекэмпа-Месси над конечными коммутативными кольцами. Срок сдачи 24.11.2022


Целью данной долгосрочной работы является написание собственной библиотеки на языке python для работы с конечными кольцами и последовательностями над ними. В качестве демонстрации использования данной библиотеки мы реализуем алгоритм Берлекэмпа-Месси над конечными коммутативными кольцами. Особенности работы
- Реализации которые мы напишем не являются эталонными в плане скорости работы. Цель, которую мы преследуем - освоить python и еще раз насладиться красотой алгебры.
- Писать код нужно на голом python, не Sage. Это позволит использовать отладку в Pycharm
- Необходимо заполнить участки кода, помеченные `YOUR CODE HERE` и подумать над местами помеченными `TODO`
- Необходимо написать собственные тесты
- Для успешной сдачи необходимо сдать заполненный jupyter блокнот и папку с проектом в pycharm.

### 1. Модуль для работы с кольцами

Сначала напишем старый добрый алгоритм Евклида.

In [3]:
def gcd_extended_base(a, b):
    # Base Case
    if a == 0:
        return b, 0, 1

    gcd, x1, y1 = gcd_extended(b % a, a)

    # Update x and y using results of recursive
    # call
    x = y1 - (b // a) * x1
    y = x1

    return gcd, x, y


def gcd_extended(a, b):
    g, u, v = gcd_extended_base(a, b)
    if g < 0:
        g, u, v = -g, -u, -v
    return g, u, v

Продемонстрируем кго работоспособность

In [4]:
def test_euclid():
    a, b = 35, -19
    g, u, v = gcd_extended(a, b)
    print(g, u, v)
    print(u * a + v * b == g)
    
test_euclid()

1 6 11
True


In [5]:
from abc import abstractmethod # для наследования
from itertools import product
from collections import defaultdict
import numpy as np

import typing as tp # для подсказки некоторых типов

Для того чтобы можно было итерироваться по кольцам то есть писать что-то типа `for r in R:` напиешем итератор кольца

In [6]:
class RingIterator:
    def __init__(self, ring):
        self.ring = ring
        self.start_idx = 0
        self.stop_idx = ring.cardinality()

        self.cur_idx = 0

    def __next__(self):
        if self.cur_idx != self.stop_idx:
            self.cur_idx += 1
            return self.ring.idx_to_elem(self.cur_idx - 1)
        else:
            raise StopIteration

Далее напишем базовый (от него будут наследоваться все кольца) класс кольца. Суть функций будет ясна из названия.

In [7]:
class Ring:
    def __call__(self, *args, **kwargs):
        raise NotImplementedError

    def is_commutative(self):
        raise NotImplementedError

    def has_identity(self):
        raise NotImplementedError

    def identity(self):
        raise NotImplementedError

    def is_finite(self):
        raise NotImplementedError

    def zero(self):
        raise NotImplementedError

    def cardinality(self):
        raise NotImplementedError

    def __iter__(self):
        return RingIterator(self)

    def idx_to_elem(self, idx):
        raise NotImplementedError

Далее напишем базовый класс элемента кольца. Для такого элемента в поле `_parent` мы будем хранить ссылку на кольцо. Отметим, что всякое кольцо является $\mathbb{Z}$-модулем поэтому уже для базового класса реализуем умножение на целые числа. При этом будем реализовывать быстрое умножение (за логарифми).

Функции, помеченные `@abstractmethod` необходимо будет перегрузить в классах наследниках

In [8]:
class RingElem:
    def __init__(self, parent):
        self._parent = parent

    def __mul_with_integer(self, n):
        ans = self._parent.zero()
        if n == 0:
            return ans

        ans = self
        cur_pow_two = self
        if n < 0:
            ans = - ans
            cur_pow_two = -cur_pow_two
            n = -n

        n = n - 1
        while n > 0:
            if (n % 2) == 1:
                ans = ans + cur_pow_two
            n = n >> 1
            cur_pow_two = cur_pow_two + cur_pow_two

        return ans

    def check_same_parents(self, other):
        return self._parent == other._parent

    @abstractmethod
    def mul_with_ring_elem(self, other):
        raise NotImplementedError

    def __mul__(self, other):
        if isinstance(other, int):
            return self.__mul_with_integer(other)

        try:
            if self.check_same_parents(other):
                return self.mul_with_ring_elem(other)
            else:
                return other.__rmul__(self)
        except AttributeError as e:
            return other.__rmul__(self)


    @abstractmethod
    def add_with_ring_elem(self, other):
        raise NotImplementedError

    def __add__(self, other):
        self.check_same_parents(other)
        return self.add_with_ring_elem(other)

    @abstractmethod
    def __neg__(self):
        pass

    def __sub__(self, other):
        return self + (-other)

    def __pow__(self, power):
        if power <= 0:
            if not self._parent.has_identity():
                assert False, 'Incorrect operation: Elements has no nonpositive power in rings without identity'

            # TODO: think about it. Maybe in domains it is reasonable
            # assert self.is_invertible(), 'Element should be invertible to have negative power'

        if power == 0:
            return self._parent.identity()

        ans = self
        cur_pow_two = self

        if power < 0:
            cur_pow_two = cur_pow_two.inverse()
            power = -power

        power = power - 1
        while power > 0:
            if (power % 2) == 1:
                ans = # YOUR CODE HERE
            cur_pow_two = # YOUR CODE HERE
            power = power >> 1

        return ans

    def __xor__(self, pow):
        return self ** pow

    def is_invertible(self):
        raise NotImplementedError

    def inverse(self):
        raise NotImplementedError

    @abstractmethod
    def __eq__(self, other):
        raise NotImplementedError

    def __ne__(self, other):
        return not self == other

    def elem_to_idx(self):
        raise NotImplementedError

Далее реализуем кольцо целых чисел по модулю.

In [19]:
class IntegerModRingElem(RingElem):
    def __init__(self, n, parent):
        super(IntegerModRingElem, self).__init__(parent)
        self.modulus = parent.modulus
        self.n = n if n >= 0 else self.modulus + n

    def elem_to_idx(self):
        return self.n

    def __eq__(self, other):
        return self._parent == other._parent and self.n == other.n

    def is_invertible(self):
        gcd, _, _ = gcd_extended(self.modulus, self.n)
        return gcd == 1

    def inverse(self):
        g, u, v = # YOUR CODE HERE
        if g != 1:
            assert False, 'Non invertible elements has no inverse'
        return self._parent(u)

    def add_with_ring_elem(self, other):
        return self._parent((self.n + other.n) % self.modulus)

    def __neg__(self):
        return self._parent((self.modulus - self.n) % self.modulus)

    def mul_with_ring_elem(self, other):
        return # YOUR CODE HERE

    def __repr__(self):
        return str(self.n)

In [20]:
class IntegerModRing(Ring):
    def __init__(self, modulus):
        self.modulus = modulus

    def cardinality(self):
        return self.modulus

    def idx_to_elem(self, idx):
        return IntegerModRingElem(idx, self)

    def zero(self):
        return IntegerModRingElem(0, self)

    def is_commutative(self):
        return True

    def has_identity(self):
        return #YOUR CODE HERE

    def identity(self):
        return IntegerModRingElem(1, self)

    def is_finite(self):
        return True

    def __repr__(self):
        return f'IntegerModRing({self.modulus})'

    def __call__(self, n):
        return IntegerModRingElem(n, self)

Продемонстрируем работу данного класса

In [21]:
def demo_zn():
    R = IntegerModRing(10)
    print('Iteration')
    for el in R:
        print(el)
        
    print('Invertible elems')
    for el in R:
        if el.is_invertible():
            print(el)
demo_zn()

Iteration
0
1
2
3
4
5
6
7
8
9
Invertible elems
1
3
7
9


Обратите внимание, что благодаря методу `__repr__` элементы печатаются необходимым образом

Продемонстрируйте работу остальных функций для данного класса.


In [22]:
#YOUR CODE HERE

Далее напишем кольцо многочленов

In [23]:
class PolynomialRingElem(RingElem):
    def __init__(self, parent, coefs: tp.List[RingElem]):
        assert len(coefs) != 0, 'list of coefficients should be non empty'
        super(PolynomialRingElem, self).__init__(parent)
        ring_zero = self._parent.base_ring.zero()

        i = len(coefs) - 1
        lead_coef = coefs[i]
        while i > 0 and lead_coef == ring_zero:
            i = i - 1
            lead_coef = coefs[i]

        coefs = [coef for coef in coefs[:i + 1]]

        if len(coefs) == 0:
            coefs = [self._parent.base_ring.zero()]
        self.coefs = coefs

    def __getitem__(self, item):
        return self.coefs[item]

    def degree(self):
        return # YOUR CODE HERE

    def __eq__(self, other):
        ans = False
        # TODO: think about this
        # if self._parent != other._parent:
        #     return ans
        if len(self.coefs) != len(other.coefs):
            return ans
        for el, el_other in zip(self.coefs, other.coefs):
            if el != el_other:
                return ans
        return True

    def mul_with_ring_elem(self, other):
        # if other is constnant from ring. Interpret it as polynomial with zero degree
        # TODO: test it
        if other._parent == self._parent.base_ring:
            other = PolynomialRingElem(self._parent, [other])

        base_ring_zero = self._parent.base_ring.zero()
        if other.degree() == -1 or self.degree() == -1:
            return self._parent([base_ring_zero])

        coefs = []
        for k in range(self.degree() + other.degree() + 1):
            coef_k = base_ring_zero
            l = max(0, k - other.degree())
            r =  #YOUR CODE HERE
            for i in range(l, r + 1):
                coef_k = coef_k + # YOUR CODE HERE
            coefs.append(coef_k)

        return self._parent(coefs)

    def add_with_ring_elem(self, other):
        if self.degree() == -1:
            return other
        elif other.degree == -1:
            return self

        if self.degree() >= other.degree():
            small = other
            big = self
        else:
            small =  #YOUR CODE HERE
            big =  #YOUR CODE HERE

        coefs = [c for c in big.coefs]
        for i, c in enumerate(small.coefs):
            coefs[i] = # YOUR CODE HERE

        return self._parent(coefs)

    def __neg__(self):
        coefs = []
        for c in self.coefs:
            coefs.append(-c)
        return self._parent( #YOUR CODE HERE)

    def leading_coef(self):
        return self.coefs[-1]

    def leading_term(self):
        d = self.degree()
        lc =  #YOUR CODE HERE
        return PolynomialRingElem(self._parent, [self._parent.zero()] * (d - 1) + [lc])

    def divide_with_rem(self, other, side):
        """
        :param other: polynomial to divide
        :param side: left or right. left: a = bq + r, right a = qb + r
        :return: quotient and remainder
        """
        assert self.check_same_parents(other), 'parents should be the same'
        lc = other.leading_coef()
        lcpol = self._parent([lc])
        lcpol_inv = self._parent([lc]) if lc == self._parent.base_ring.identity() else self._parent([lc ** -1])
        other = lcpol_inv * other if side == 'left' else other * lcpol_inv  # for workin wiht monic pols

        r = self
        q = self._parent.zero()
        while r.degree() >= other.degree():
            rd = r.degree()
            od = other.degree()
            pol_to_mul = PolynomialRingElem(self._parent,
                                            [self._parent.base_ring.zero()] * (rd - od) + [r.leading_coef()])
            q = q + # YOUR CODE HERE
            if side == 'left':
                vich = other * pol_to_mul if side == 'left' else  #YOUR CODE HERE
                vich = -vich
                r = r + # YOUR CODE HERE

        q = lcpol * q if side == 'left' else q * lcpol
        return q, r

    def __repr__(self):
        degree = self.degree()
        if degree <= 0:
            return f'{self.coefs[0]}'

        coefrepr = self.leading_coef().__repr__()
        if '+' in coefrepr and not coefrepr.endswith(')'):
            coefrepr = f'({coefrepr})'

        ans = f'{coefrepr}{self._parent.gen_name}'
        if degree != 1:
            ans += f'^{degree}'

        zero = self._parent.base_ring.zero()
        for d in range(degree - 1, -1, -1):
            coef = self.coefs[d]
            if coef != zero:
                coefrepr = coef.__repr__()
                if '+' in coefrepr and not coefrepr.endswith(')'):
                    coefrepr = f'({coefrepr})'
                if d not in [0, 1]:
                    ans += f' + {coefrepr}{self._parent.gen_name}^{d}'
                elif d == 1:
                    ans += f' + {coefrepr}{self._parent.gen_name}'
                elif d == 0:
                    ans += f' + {coefrepr}'
        if len(ans) == 0:
            return 0
        return ans

    def is_monic(self):
        return self.leading_coef() ==  # YOUR CODE HERE

    def elem_to_idx(self):
        assert self._parent.base_ring.is_finite(), 'to take index ring should be finite'
        ans = 0
        card = self._parent.base_ring.cardinality()
        for i, c in enumerate(self.coefs):
            ans += c.elem_to_idx() * (card ** i)
        return ans


In [24]:
class PolynomialRing(Ring):
    def __init__(self, base_ring: Ring, gen_name='x'):
        self.base_ring = base_ring
        self.gen_name = gen_name
        self.base_ring_card = base_ring.cardinality()

    def is_finite(self):
        return False

    def __call__(self, coefs: tp.List[RingElem]):
        return PolynomialRingElem(self, coefs)

    def gen(self):
        e = self.base_ring.identity()
        zero = self.base_ring.zero()
        return self([zero, e])

    def idx_to_elem(self, idx):
        assert self.base_ring.is_finite(), 'Base ring should be finite'
        if idx == 0:
            return PolynomialRingElem(self, [self.base_ring.zero()])
        coefs = []
        while idx > 0:
            base_ring_idx = idx % self.base_ring_card
            coef = self.base_ring.idx_to_elem(base_ring_idx)
            coefs.append(coef)
            idx = idx // self.base_ring_card
        return PolynomialRingElem(self, coefs)

    def is_commutative(self):
        return self.base_ring.is_commutative()

    def has_identity(self):
        return self.base_ring.has_identity()

    def zero(self):
        return PolynomialRingElem(self, [self.base_ring.zero()])

    def identity(self):
        return PolynomialRingElem(self, [self.base_ring.identity()])

    def cardinality(self):
        return -1 # для ненулевого базового кольца кольцо многочленов бесконечно

Продемонстрируем работу данного кольца

In [28]:
def demo_pols():
    z3 = IntegerModRing(3)
    z3x = PolynomialRing(z3, 'x')
    f = z3x([z3(2), z3(1)])
    g = z3x([z3(0), z3(1), z3(2)])
    print(f'f={f}\ng={g}\nf*g = {f * g}\nf+g = {f + g}')

    print('-' * 50)

    z3 = IntegerModRing(3)
    z3x = PolynomialRing(z3, 'y')
    f = z3x([z3(0), z3(2)])
    print(f.degree())
    g = z3x([z3(0), z3(1), z3(2)])
    print(f'f={f}\ng={g}\nf*g = {f * g}\nf+g = {f + g}')
    
    
    print('-' * 50)
    z3 = IntegerModRing(3)
    z3x = PolynomialRing(z3, 'y')
    f = z3x([z3(1)])
    g = z3x([z3(1)])
    q, r = f.divide_with_rem(g, 'left')
    print(f'f={f}\ng={g}\nq = {q}\nr = {r}')

    print('-' * 50)
    z3 = IntegerModRing(3)
    z3x = PolynomialRing(z3, 'y')
    f = z3x([z3(1)])
    g = z3x([z3(1), z3(1), z3(2)])
    q, r = f.divide_with_rem(g, 'left')
    print(f'f={f}\ng={g}\nq = {q}\nr = {r}')

    print('-' * 50)
    z3 = IntegerModRing(3)
    z3x = PolynomialRing(z3, 'y')
    f = z3x([z3(1), z3(1), z3(2)])
    g = z3x([z3(1)])
    q, r = f.divide_with_rem(g, 'left')
    print(f'f={f}\ng={g}\nq = {q}\nr = {r}')

    print('-' * 50)
    z2 = IntegerModRing(2)
    z2x = PolynomialRing(z2, 'y')
    f = z2x([z2(1), z2(1), z2(1)])
    g = z2x([z2(1), z2(1)])
    q, r = f.divide_with_rem(g, 'left')
    print(f'f={f}\ng={g}\nq = {q}\nr = {r}')

    print('-' * 50)
    z2 = IntegerModRing(6)
    z2x = PolynomialRing(z2, 'y')
    x = z2x([z2(0), z2(1)])
    e = z2x([z2(1)])
    f = x ** 70 - e
    g = x ** 3 + x ** 2 + e
    q, r = f.divide_with_rem(g, 'left')
    print(f'f={f}\ng={g}\nq = {q}\nr = {r}')
    
demo_pols()

f=1x + 2
g=2x^2 + 1x
f*g = 2x^3 + 2x^2 + 2x
f+g = 2x^2 + 2x + 2
--------------------------------------------------
1
f=2y
g=2y^2 + 1y
f*g = 1y^3 + 2y^2
f+g = 2y^2
--------------------------------------------------
f=1
g=1
q = 1
r = 0
--------------------------------------------------
f=1
g=2y^2 + 1y + 1
q = 0
r = 1
--------------------------------------------------
f=2y^2 + 1y + 1
g=1
q = 2y^2 + 1y + 1
r = 0
--------------------------------------------------
f=1y^2 + 1y + 1
g=1y + 1
q = 1y
r = 1
--------------------------------------------------
f=1y^70 + 5
g=1y^3 + 1y^2 + 1
q = 1y^67 + 5y^66 + 1y^65 + 4y^64 + 3y^63 + 2y^62 + 3y^60 + 1y^59 + 5y^58 + 4y^57 + 1y^56 + 2y^54 + 3y^53 + 3y^52 + 1y^51 + 2y^50 + 1y^49 + 4y^48 + 5y^46 + 3y^45 + 3y^44 + 4y^43 + 5y^42 + 4y^41 + 4y^40 + 3y^39 + 5y^38 + 3y^37 + 1y^35 + 2y^34 + 4y^33 + 1y^32 + 3y^31 + 5y^30 + 3y^28 + 4y^27 + 2y^26 + 1y^25 + 1y^24 + 3y^23 + 2y^22 + 3y^21 + 4y^19 + 5y^18 + 1y^17 + 1y^16 + 5y^14 + 1y^11 + 5y^10 + 1y^9 + 4y^8 + 3y^7 + 2

Продемонстрируйте работу остальных функций класса. Напишите тесты.

#YOUR CODE HERE

Далее напишем класс идеала 

In [30]:
class IdealOverRing:
    def __init__(self, ring, gens, name=None):
        self.gens = gens
        self.ring = ring
        self.name = name

    def add_gen(self, gen):
        self.gens.append(gen)

    def reduce(self, elem):
        raise NotImplementedError

    def contains(self, elem):
        assert self.ring.is_finite(), 'Generic check containment realized only for finite rings'
        n_gens = len(self.gens)
        for coefs in product(*[self.ring] * n_gens): # TODO: think about this elegant construction
            prods = [coef * gen for coef, gen in zip(coefs, self.gens)]
            res = self.ring.zero()
            for p in prods:
                res = res + p
            if res == elem:
                return True, coefs
        return False, None

    def __repr__(self):
        return self.name

Продемонстрируем его работу

In [31]:
def test_containment():
    R = IntegerModRing(6)
    I = IdealOverRing(R, [R(2), R(3)])
    elem = R(5)
    ans, els = I.contains(elem)
    print(ans, els)
    
test_containment()

True (1, 1)


Продемонстрируйте работу остальных функций класса. Напишите тесты.

#YOUR CODE HERE

Напишем класс главного идеала над полиномиальным кольцом

In [33]:
class PrincipalIdealOverPolynomialRing(IdealOverRing):
    def __init__(self, ring, gens: tp.List[PolynomialRingElem], is_left: bool, name=None):
        super(PrincipalIdealOverPolynomialRing, self).__init__(ring, gens, name)
        assert len(gens) == 1, 'Number of generators for principal ideal should be equal to one'
        gen = gens[0]
        assert gen.is_monic(), 'Ideal should be generated by monic polynomial'
        self.gen = gen
        self.is_left = is_left

    def reduce(self, elem: PolynomialRingElem):
        if self.is_left:
            _, ans = elem.divide_with_rem(self.gen, side='left')
        else:
            _, ans = # YOUR CODE HERE
        return ans

    def contains(self, elem):
        red = self.reduce(elem)
        # YOUR CODE HERE

Продемонстрируйте работу  функций класса. Напишите тесты.

#YOUR CODE HERE

Осталось чуть-чуть. Напишем класс Фактор-кольца

In [35]:
class FactorRingElem(RingElem):
    def __init__(self, parent, elem, need_reduce=True):
        super(FactorRingElem, self).__init__(parent)
        self.ideal = parent.ideal
        self.elem = self.ideal.reduce(elem) if need_reduce else elem

    def mul_with_ring_elem(self, other):
        ans = self.elem * other.elem
        ans = self.ideal.reduce(ans)
        return FactorRingElem(self._parent, ans, need_reduce=False)

    def add_with_ring_elem(self, other):
        # YOUR CODE HERE

    def __neg__(self):
        # YOUR CODE HERE
        
    def __eq__(self, other):
        return self.elem == other.elem

    def is_monic(self):
        return self.elem.is_monic()

    def __repr__(self):
        return f'{self.elem.__repr__()} + {self.ideal.name}'

    def elem_to_idx(self):
        return self.elem.elem_to_idx()


class FactorRing(Ring):
    def __init__(self, ring, ideal: IdealOverRing):
        self.ring = ring
        self.ideal = ideal

    def __call__(self, *args, **kwargs):
        elem = self.ring(*args, **kwargs)
        return FactorRingElem(self, elem)

    def zero(self):
        return FactorRingElem(self, self.ring.zero(), need_reduce=False)

    def idx_to_elem(self, idx):
        assert 0 <= idx < self.cardinality(), 'idx is not correct'
        elem = self.ring.idx_to_elem(idx)
        return FactorRingElem(self, elem, need_reduce=True)


class FactorRingOverPrincipalPolynomialIdeal(FactorRing):
    def __init__(self, ring, ideal):
        super(FactorRingOverPrincipalPolynomialIdeal, self).__init__(ring, ideal)

    def cardinality(self):
        return self.ring.base_ring.cardinality() ** self.ideal.gens[0].degree()

    def identity(self):
        e = self.ring.identity()
        return FactorRingElem(self, e)

    def is_finite(self):
        if self.ring.base_ring.is_finite():
            return True

    def has_identity(self):
        return self.ring.has_identity()

Продемонстрируем, что у нас получилось

Построим поле из 4 элементов ~~Леша Hello~~ как фактор-кольцо

In [40]:
def test_factor_rings():
    R = IntegerModRing(2)
    Rx = PolynomialRing(R, gen_name='\u03b2') # не пугайтесь это beta
    x = Rx.gen()
    pol = Rx([R(1), R(1), R(1)])
    I = PrincipalIdealOverPolynomialRing(Rx, [pol], is_left=True, name='I')
    RxI = FactorRingOverPrincipalPolynomialIdeal(Rx, I)
    for el in RxI:
        print(el)
test_factor_rings()

0 + I
1 + I
1β + I
1β + 1 + I


Построим более интересную штуку GF(16), дважды построив фактор-кольцо

In [46]:
def test_factor_rings():
    R = IntegerModRing(2)
    Rx = PolynomialRing(R, gen_name='\u03b2')
    x = Rx.gen()
    pol = Rx([R(1), R(1), R(1)])
    I = PrincipalIdealOverPolynomialRing(Rx, [pol], is_left=True, name='I')
    RxI = FactorRingOverPrincipalPolynomialIdeal(Rx, I)
   
    beta = RxI([R(0), R(1)])
    e_RxI = RxI.identity()
    RxIy = PolynomialRing(RxI, gen_name='y')
    y = RxIy.gen()
    
    poly = RxIy([beta, beta, e_RxI]) # неприводимый многочлен над GF(4)
    
    J = PrincipalIdealOverPolynomialRing(RxIy, [poly], is_left=True, name='J')
    RxIyJ = FactorRingOverPrincipalPolynomialIdeal(RxIy, J)
    
    for i, el in enumerate(RxIyJ):
        print(i, ' ', el)
test_factor_rings()

0   0 + I + J
1   1 + I + J
2   1β + I + J
3   1β + 1 + I + J
4   (1 + I)y + J
5   (1 + I)y + (1 + I) + J
6   (1 + I)y + (1β + I) + J
7   (1 + I)y + (1β + 1 + I) + J
8   (1β + I)y + J
9   (1β + I)y + (1 + I) + J
10   (1β + I)y + (1β + I) + J
11   (1β + I)y + (1β + 1 + I) + J
12   (1β + 1 + I)y + J
13   (1β + 1 + I)y + (1 + I) + J
14   (1β + 1 + I)y + (1β + I) + J
15   (1β + 1 + I)y + (1β + 1 + I) + J


Построим таблички Кэли для GF(16) используя индексы элементов

In [54]:
def test_factor_rings():
    R = IntegerModRing(2)
    Rx = PolynomialRing(R, gen_name='\u03b2')
    x = Rx.gen()
    pol = Rx([R(1), R(1), R(1)])
    I = PrincipalIdealOverPolynomialRing(Rx, [pol], is_left=True, name='I')
    RxI = FactorRingOverPrincipalPolynomialIdeal(Rx, I)


    beta = RxI([R(0), R(1)])
    e_RxI = RxI.identity()
    RxIy = PolynomialRing(RxI, gen_name='y')
    y = RxIy.gen()
    poly = RxIy([beta, beta, e_RxI])
    J = PrincipalIdealOverPolynomialRing(RxIy, [poly], is_left=True, name='J')
    RxIyJ = FactorRingOverPrincipalPolynomialIdeal(RxIy, J)

    dct_idx_to_elem = {}
    
    for i, el in enumerate(RxIyJ):
        print(i, ':', el, "idx:", el.elem_to_idx())
        dct_idx_to_elem[i] = el
        

    table_pl = []
    table_pr = []

    for i in range(16):
        table_pl.append([])
        table_pr.append([])
        for j in range(16):
            idx_pl = (dct_idx_to_elem[i] + dct_idx_to_elem[j]).elem_to_idx()
            idx_pr = (dct_idx_to_elem[i] * dct_idx_to_elem[j]).elem_to_idx()
            table_pl[i].append(idx_pl)
            table_pr[i].append(idx_pr)

    print()
    for i in range(16):
        print(table_pr[i])

test_factor_rings()

0 : 0 + I + J idx: 0
1 : 1 + I + J idx: 1
2 : 1β + I + J idx: 2
3 : 1β + 1 + I + J idx: 3
4 : (1 + I)y + J idx: 4
5 : (1 + I)y + (1 + I) + J idx: 5
6 : (1 + I)y + (1β + I) + J idx: 6
7 : (1 + I)y + (1β + 1 + I) + J idx: 7
8 : (1β + I)y + J idx: 8
9 : (1β + I)y + (1 + I) + J idx: 9
10 : (1β + I)y + (1β + I) + J idx: 10
11 : (1β + I)y + (1β + 1 + I) + J idx: 11
12 : (1β + 1 + I)y + J idx: 12
13 : (1β + 1 + I)y + (1 + I) + J idx: 13
14 : (1β + 1 + I)y + (1β + I) + J idx: 14
15 : (1β + 1 + I)y + (1β + 1 + I) + J idx: 15

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
[0, 2, 3, 1, 8, 10, 11, 9, 12, 14, 15, 13, 4, 6, 7, 5]
[0, 3, 1, 2, 12, 15, 13, 14, 4, 7, 5, 6, 8, 11, 9, 10]
[0, 4, 8, 12, 10, 14, 2, 6, 15, 11, 7, 3, 5, 1, 13, 9]
[0, 5, 10, 15, 14, 11, 4, 1, 7, 2, 13, 8, 9, 12, 3, 6]
[0, 6, 11, 13, 2, 4, 9, 15, 3, 5, 8, 14, 1, 7, 10, 12]
[0, 7, 9, 14, 6, 1, 15, 8, 11, 12, 2, 5, 13, 10, 4, 3]
[0, 8, 12, 4, 15, 7, 3, 11, 5, 13, 9, 1, 10

В общем с этим можно уже некторое время поиграть =)

Напишите тесты для остальных функций фактор-кольца

In [55]:
# YOUR CODE HERE

# 2. Далее напишем модуль для работы с последовательностями

In [57]:
class SequenceOverRing:
    def __init__(self, terms: tp.List, ring):
        self.terms = terms
        self.ring = ring
        self.ring_zero = self.ring.zero()
        self.period = None

    def __len__(self):
        return len(self.terms)

    def len(self):
        return len(self)

    def cnt_leading_zeros(self):
        ans = 0
        for i in range(len(self)):
            if self[i] == self.ring_zero:
                ans += 1
            else:
                break
        return ans

    def __getitem__(self, idx):
        return self.terms[idx]

    def __rmul__(self, pol: PolynomialRingElem):
        old_len = len(self)
        new_len = old_len - pol.degree()
        if new_len <= 0:
            return SequenceOverRing([], self.ring)
        new_terms = []
        for i in range(new_len):
            res = self.ring_zero
            for j in range(pol.degree() + 1):
                res = res + pol[j] * self.terms[i + j]
            new_terms.append(res)
        return SequenceOverRing(new_terms, self.ring)

    def __add__(self, other):
        new_len = min(self.len(), other.len())
        terms = []
        for i in range(new_len):
            terms.append(self[i] + other[i])
        return SequenceOverRing(terms, self.ring)

    def __neg__(self):
        return SequenceOverRing([-el for el in self.terms], self.ring)

    def is_empty(self):
        return len(self) == 0

    def is_zero(self):
        for term in self.terms:
            if term != self.ring_zero:
                return False
        return True

    def __sub__(self, other):
        terms = []
        for i in range(min(len(self), len(other))):
            terms[i] = self.terms[i] - other.terms[i]
        return SequenceOverRing(terms, self.ring)

    def check_is_period(self, t):
        ans = False
        n = len(self)
        for i in range(n - t):
            if self[i] != self[i + t]:
                return ans
        return True

    def check_is_minimal_period(self, t, t_proper_divs):
        if not self.check_is_period(t):
            return False
        ans = False
        for d in t_proper_divs:
            if self.check_is_period(d):
                return ans
        return True

    def compute_period(self, lam=0):
        """
        TODO: make for non zero lambda
        """
        n = len(self)
        ans = -1
        for per in range(1, len(self)):
            # print(f'try {per}')
            was_break = False
            for i in range(n - per):
                if self[i] != self[i + per]:
                    was_break = True
                    break
            if not was_break:
                ans = per
                self.period = per
                return ans

    def __repr__(self):
        ans = ''
        for i in range(10):
            ans += f'{self[i].__repr__()} '
        return ans


class LRSoverRing(SequenceOverRing):
    def __init__(self, ring, init_vector, char_pol):
        super(LRSoverRing, self).__init__(terms=[el for el in init_vector], ring=ring)
        assert char_pol.is_monic(), 'characteristic polynomial should be monic'
        assert len(init_vector) == char_pol.degree()
        self.init_vector = init_vector
        self.char_pol = char_pol

    def compute_n_terms(self, n):
        for i in range(n):
            res = self.ring_zero
            for j in range(self.char_pol.degree()):
                res = res - self.char_pol[j] * self.terms[i + j]
            self.terms.append(res)

С данным классом вы уже знакому. Обратите внимание на функцию `__repr__` 

А также класс для скрученных ЛРП

In [58]:
class RingAutomorplism:
    def __init__(self, ring):
        self.ring = ring

    def __call__(self, ring_elem):
        raise NotImplementedError


class GaloisRingAutomorphism(RingAutomorplism):
    def __init__(self, galois_ring, galois_ring_invariant):
        super(GaloisRingAutomorphism, self).__init__(galois_ring)
        self.galois_ring_invariant = galois_ring_invariant

    def __call__(self, ring_elem):
        raise NotImplementedError


class FiniteFieldAutomorphism(GaloisRingAutomorphism):
    def __init__(self, finite_field, finite_field_invariant):
        super(FiniteFieldAutomorphism, self).__init__(finite_field, finite_field_invariant)

    def __call__(self, ring_elem):
        return ring_elem ** (self.galois_ring_invariant.cardinality())


class SkewLRS(SequenceOverRing):
    def __init__(self, ring, init_vec, char_pol_table: tp.List[tp.List], sigma):
        super(SkewLRS, self).__init__(terms=init_vec, ring=ring)
        self.sigma = sigma
        self.ring_ivariant = self.sigma.galois_ring_invariant
        self.char_pol_table = char_pol_table

    def show_char_pol_table(self):
        for row in self.char_pol_table:
            for el in row:
                print(el, end=' ')
            print()

    def compute_n_terms(self, n_terms):
        m = len(self.char_pol_table)
        for i in range(n_terms):
            res = self.ring_zero
            for j in range(m):
                sigma_pow = self[i + j]
                n = len(self.char_pol_table[j])
                for k in range(n):
                    res = res - self.char_pol_table[j][k] * sigma_pow
                    sigma_pow = self.sigma(sigma_pow)
            self.terms.append(res)

# 3. Берлекэмп Месси

Сначала напишем вспомогательную функцию для разбиения кольца на классы ассоциир элементов

In [59]:
def split_ring_left_assoc_classes(ring):
    e = ring.identity()
    dct_idx_elem = {}
    identity_idx = -1
    for i, el in enumerate(ring):
        dct_idx_elem[i] = el
        if el == e:
            identity_idx = i

    card = ring.cardinality()
    invertible_elems_idxs = set()
    for i in range(card):
        for j in range(card):
            if dct_idx_elem[i] * dct_idx_elem[j] == # YOUR CODE HERE:
                invertible_elems_idxs.add(i)

    classes = []
    reprs = []

    classes.append(invertible_elems_idxs)
    reprs.append(identity_idx)

    elem_idxs = set(range(card)) - set(invertible_elems_idxs)
    while len(elem_idxs) != 0:
        idx = next(iter(elem_idxs))
        if idx == 0:
            elem_idxs.remove(idx)
            continue
        reprs.append(idx)
        cur_cls = set()
        cur_elem = dct_idx_elem[idx]
        for inv_el_idx in invertible_elems_idxs:
            prod = dct_idx_elem[inv_el_idx] * # YOUR CODE HERE
            cur_cls.add(prod.elem_to_idx())
        classes.append(cur_cls)
        for idx in cur_cls:
            elem_idxs.remove(idx)

    classes = [[dct_idx_elem[j] for j in cls] for cls in classes]
    reprs = [dct_idx_elem[j] for j in  #YOUR CODE HERE]
    return classes, reprs

Протестируем данную функцию

In [64]:
def test_split():
    def split_z(n):
        R = IntegerModRing(n)
        clss, reprs = split_ring_left_assoc_classes(R)
        print('-' * 50)
        print(clss)
        print(reprs)

    def split_gf4():
        R = IntegerModRing(2)
        Rx = PolynomialRing(R)
        pol = Rx([R(1), R(1), R(1)])
        I = PrincipalIdealOverPolynomialRing(Rx, [pol], name='I', is_left=True)
        RxI = FactorRingOverPrincipalPolynomialIdeal(Rx, I)

        clss, reprs = split_ring_left_assoc_classes(RxI)
        print('-' * 50)
        print(clss)
        print(reprs)

    def split_gf16():
        R = IntegerModRing(2)
        Rx = PolynomialRing(R, gen_name='\u03b2')
        pol = Rx([R(1), R(1), R(1)])
        I = PrincipalIdealOverPolynomialRing(Rx, [pol], is_left=True, name='I')
        RxI = FactorRingOverPrincipalPolynomialIdeal(Rx, I)

        beta = RxI([R(0), R(1)])
        e_RxI = RxI.identity()
        RxIy = PolynomialRing(RxI, gen_name='y')
        poly = RxIy([beta, beta, e_RxI])
        J = PrincipalIdealOverPolynomialRing(RxIy, [poly], is_left=True, name='J')
        RxIyJ = FactorRingOverPrincipalPolynomialIdeal(RxIy, J)
        clss, reprs = split_ring_left_assoc_classes(RxIyJ)
        print('-' * 50)
        print(clss)
        print(reprs)

    split_z(8)
    split_z(100)
    split_gf4()
    split_gf16()


test_split()

--------------------------------------------------
[[1, 3, 5, 7], [2, 6], [4]]
[1, 2, 4]
--------------------------------------------------
[[1, 3, 7, 9, 11, 13, 17, 19, 21, 23, 27, 29, 31, 33, 37, 39, 41, 43, 47, 49, 51, 53, 57, 59, 61, 63, 67, 69, 71, 73, 77, 79, 81, 83, 87, 89, 91, 93, 97, 99], [2, 6, 14, 18, 22, 26, 34, 38, 42, 46, 54, 58, 62, 66, 74, 78, 82, 86, 94, 98], [4, 8, 12, 16, 24, 28, 32, 36, 44, 48, 52, 56, 64, 68, 72, 76, 84, 88, 92, 96], [65, 35, 5, 45, 15, 85, 55, 95], [70, 10, 90, 30], [40, 80, 20, 60], [25, 75], [50]]
[1, 2, 4, 5, 10, 20, 25, 50]
--------------------------------------------------
[[1 + I, 1x + I, 1x + 1 + I]]
[1 + I]
--------------------------------------------------
[[1 + I + J, 1β + I + J, 1β + 1 + I + J, (1 + I)y + J, (1 + I)y + (1 + I) + J, (1 + I)y + (1β + I) + J, (1 + I)y + (1β + 1 + I) + J, (1β + I)y + J, (1β + I)y + (1 + I) + J, (1β + I)y + (1β + I) + J, (1β + I)y + (1β + 1 + I) + J, (1β + 1 + I)y + J, (1β + 1 + I)y + (1 + I) + J, (1β + 1 + 

Ну и наконец то ради чего мы все здесь собрались

In [65]:
class BerlecampMasseyVLOverFCR:
    def __init__(self, reprs=None, verbose=False):
        self.verbose = verbose
        self.reprs = reprs

        self.init_seq = None

        self.ring = None
        self.pol_ring = None
        self.ideals: tp.Dict[tp.Tuple[int, int], IdealOverRing] = {}
        self.ideals_inf_table: tp.Dict[tp.Tuple[int, int], tp.List[tp.Dict[str, int]]] = {}

        self.n_tables = 0

        self.hist: tp.List[tp.List[tp.Dict]] = []
        self.hist_step: tp.List[tp.List[tp.Dict]] = []

    def clean(self):
        self.init_seq = None
        self.ideals = {}
        self.hist = []
        self.hist_step = []
        self.n_tables = 0

    def initialize(self, init_seq):
        old_ring = self.ring
        self.ring = init_seq.ring
        if self.reprs is None or old_ring != self.ring:
            if self.verbose:
                print(f'Split ring {self.ring} to left assoc classes')
            _, reprs = # YOUR CODE HERE
            self.reprs = reprs
            print(f'Cnt classes {len(reprs)}')
        self.init_seq = init_seq
        self.n_tables = len(self.reprs)
        self.pol_ring = PolynomialRing(self.ring)

        for _ in range(self.n_tables):
            self.hist.append([])
            self.hist_step.append([])

        self.ideals = defaultdict(lambda: IdealOverRing(self.ring, []))
        self.ideals_inf_table = defaultdict(list)

    def fit(self, init_seq: SequenceOverRing):
        self.clean()
        self.initialize(init_seq)
        is_bm_stop = False
        idx_step = 0
        while not is_bm_stop:
            is_bm_stop = self.make_step(idx_step)
            if self.verbose:
                self.print_step_info(idx_step)
            idx_step += 1
        if self.verbose:
            print(f'{self.hist[0][-1]["g"]}')

    def make_step(self, idx_step):
        if self.verbose:
            print(f'Step number {idx_step}')
        if idx_step == 0:
            return self.zero_step()
        else:
            return self.step(idx_step)

    def zero_step(self):
        bm_stop = False
        for idx_table, r in enumerate(self.reprs):
            g0 = self.pol_ring([r])
            self.update_inf(idx_table, g0)
            self.update_inf_step(idx_table)

            if self.is_finished(idx_table):
                if idx_table == 0:
                    bm_stop = True
                    break
        self.compute_ideals(idx_step=0)
        return bm_stop

    def step(self, idx_step):
        is_bm_stop = self.fill_tables(idx_step)
        if not is_bm_stop:
            self.compute_ideals(idx_step)
        return  #YOUR CODE HERE

    def is_usn_in_ideal(self, n, idx_step):
        ksn = self.hist[n][-1]['k']
        usn_ksn = # YOUR CODE HERE
        assert usn_ksn != self.ring.zero()
        I = self.ideals[idx_step - 1, ksn]
        return I.contains(usn_ksn)

    def fill_tables(self, idx_step):
        x = self.pol_ring.gen()
        bm_stop = False
        for n in range(self.n_tables):
            if self.is_finished(n):
                continue

            g = # YOUR CODE HERE
            self.update_inf(idx_table=n, g=g)

            if self.is_finished(n):
                self.update_inf_step(idx_table=n)
                if n == 0:
                    bm_stop = True
                    return bm_stop
                else:
                    continue

            is_in_ideal, coefs = self.is_usn_in_ideal(n, idx_step)
            while not self.is_finished(n) and is_in_ideal:
                Fsn = self.hist[n][-1]['g']
                ksn = self.hist[n][-1]['k']
                for i, c in enumerate(coefs):
                    ni = self.ideals_inf_table[idx_step - 1, ksn][i]['idx_table']
                    si = # YOUR CODE HERE
                    cpol = self.pol_ring([c])
                    Fsn = Fsn - cpol * # YOUR CODE HERE
                self.update_inf(n, Fsn)
                if not self.is_finished(n):
                    is_in_ideal, coefs = self.is_usn_in_ideal(n, idx_step)

            if self.is_finished(n):
                self.update_inf_step(n)
                if n == 0:
                    bm_stop = True
                    return bm_stop
                else:
                    continue
            else:
                self.update_inf_step(n)
        return bm_stop

    def compute_ideals(self, idx_step):
        if idx_step != 0:
            for k in range(len(self.init_seq) - idx_step):
                Isk = self.ideals[idx_step, k]
                Is_1k = self.ideals[idx_step - 1, k]
                prev_inf = self.ideals_inf_table[idx_step - 1, k]
                for gen in Is_1k.gens:
                    Isk.add_gen(gen)
                self.ideals_inf_table[idx_step, k].extend(prev_inf)

        for n in range(self.n_tables):
            if self.is_finished(n):
                continue
            k = self.hist_step[n][idx_step]['k']
            utnk =  #YOUR CODE HERE
            Isk = # YOUR CODE HERE
            if not Isk.contains(utnk)[0]:
                Isk.add_gen(utnk)
                self.ideals_inf_table[idx_step, k].append({'idx_step': idx_step, 'idx_table': n})

    def is_finished(self, idx_table):
        l = self.hist[idx_table][-1]['k'] + self.hist[idx_table][-1]['m']
        if l == # YOUR CODE HERE:
            return True
        return False

    def update_inf(self, idx_table, g):
        m = g.degree()
        u = g * self.init_seq
        k = u.cnt_leading_zeros()
        self.hist[idx_table].append({'g': g, 'k': k, 'm': m, 'u': u})

    def update_inf_step(self, idx_table):
        self.hist_step[idx_table].append(self.hist[idx_table][-1])

    def print_step_info(self, idx_step):
        print(f'After step idx {idx_step}:')
        for n in range(self.n_tables):
            print(f'table {n}')
            if idx_step >= len(self.hist_step[n]):
                idx_step = len(self.hist_step[n]) - 1
            for el in ['g', 'k', 'm']:
                print(f'{el}_{idx_step}: {self.hist_step[n][idx_step][el]}')
            print(f'u_{idx_step}: {self.hist_step[n][idx_step]["u"].terms}')
            print()

Протестируем на примерах из книги В.Л.Куракина. Обратите внимание, что алгоритм прекрасно работает над полем

In [68]:
def test_bm_fcr():
    R = IntegerModRing(6)
    terms = [2, 3, 4, 1, 4, 0, 0, 5, 0, 5]
    terms = [R(el) for el in terms]
    u = SequenceOverRing(terms, R)

    bm_fcr = BerlecampMasseyVLOverFCR(verbose=True)
    bm_fcr.fit(u)

    R = IntegerModRing(6)
    terms = [3, 4, 1, 1, 3, 1, 4, 4, 0, 1]
    terms = [R(el) for el in terms]
    u = SequenceOverRing(terms, R)

    bm_fcr = BerlecampMasseyVLOverFCR(verbose=True)
    bm_fcr.fit(u)

    R = IntegerModRing(7)
    terms = [1, 0, 0, 2, 1, 0, 4, 4, 1, 1]
    terms = [R(el) for el in terms]
    u = SequenceOverRing(terms, R)

    bm_fcr = BerlecampMasseyVLOverFCR(verbose=True)
    bm_fcr.fit(u)

    R = IntegerModRing(5)
    terms = [2, 4, 3, 4, 4, 0, 4, 0, 2, 4]
    terms = [R(el) for el in terms]
    u = SequenceOverRing(terms, R)

    bm_fcr = BerlecampMasseyVLOverFCR(verbose=True)
    bm_fcr.fit(u)
    
test_bm_fcr()

Split ring IntegerModRing(6) to left assoc classes
Cnt classes 3
Step number 0
After step idx 0:
table 0
g_0: 1
k_0: 0
m_0: 0
u_0: [2, 3, 4, 1, 4, 0, 0, 5, 0, 5]

table 1
g_0: 2
k_0: 0
m_0: 0
u_0: [4, 0, 2, 2, 2, 0, 0, 4, 0, 4]

table 2
g_0: 3
k_0: 1
m_0: 0
u_0: [0, 3, 0, 3, 0, 0, 0, 3, 0, 3]

Step number 1
After step idx 1:
table 0
g_1: 1x
k_1: 0
m_1: 1
u_1: [3, 4, 1, 4, 0, 0, 5, 0, 5]

table 1
g_1: 2x
k_1: 1
m_1: 1
u_1: [0, 2, 2, 2, 0, 0, 4, 0, 4]

table 2
g_1: 3x
k_1: 0
m_1: 1
u_1: [3, 0, 3, 0, 0, 0, 3, 0, 3]

Step number 2
After step idx 2:
table 0
g_2: 1x^2 + 2x + 1
k_2: 2
m_2: 2
u_2: [0, 0, 4, 3, 4, 5, 4, 4]

table 1
g_2: 2x^2 + 4x + 2
k_2: 2
m_2: 2
u_2: [0, 0, 2, 0, 2, 4, 2, 2]

table 2
g_2: 3x^2 + 3
k_2: 3
m_2: 2
u_2: [0, 0, 0, 3, 0, 3, 0, 0]

Step number 3
After step idx 3:
table 0
g_3: 1x^3 + 2x^2 + 3x
k_3: 2
m_3: 3
u_3: [0, 0, 5, 0, 5, 4, 2]

table 1
g_3: 2x^3 + 4x + 2
k_3: 7
m_3: 3
u_3: [0, 0, 0, 0, 0, 0, 0]

table 2
g_3: 3x^3 + 3x
k_3: 2
m_3: 3
u_3: [0, 0, 3, 0, 3, 0, 0]



Напишем стресс тест для импульсных ЛРП

In [73]:
def test_bm_lrs_random():
    print('-' * 100)
    print('test bm lrs')

    bm = BerlecampMasseyVLOverFCR(verbose=False)
   

    for i in range(10):
        n = np.random.randint(2, 200)
        R = IntegerModRing(n)
        S = PolynomialRing(R)
        x = S.gen()
        degree = np.random.randint(2, 30)
        char_pol = x ** degree + S([R(el) for el in np.random.randint(0, n - 1, degree - 1)])
        seq = LRSoverRing(ring=R, init_vector=[R(0)] * (degree - 1) + [R(1)], char_pol=char_pol)
        seq.compute_n_terms(2 * degree)
        bm.fit(seq)
        assert bm.hist[0][-1]['g'] == char_pol, 'Something wrong'
        print(bm.hist[0][-1]['g'] == char_pol)
        print(f'Find char pol \n{bm.hist[0][-1]["g"]}')
    print('-' * 100)

test_bm_lrs_random()

----------------------------------------------------------------------------------------------------
test bm lrs
Cnt classes 3
True
Find char pol 
1x^19 + 64x^17 + 74x^16 + 131x^15 + 50x^14 + 119x^13 + 34x^12 + 99x^11 + 109x^10 + 21x^9 + 63x^8 + 82x^7 + 41x^6 + 35x^5 + 130x^4 + 74x^3 + 51x^2 + 42x + 122
Cnt classes 3
True
Find char pol 
1x^6 + 28x^4 + 3x^3 + 25x^2 + 33x + 48
Cnt classes 3
True
Find char pol 
1x^21 + 158x^19 + 14x^18 + 94x^17 + 66x^16 + 120x^15 + 164x^14 + 131x^13 + 58x^12 + 156x^11 + 2x^10 + 129x^9 + 36x^8 + 16x^7 + 143x^6 + 170x^5 + 169x^4 + 127x^3 + 98x^2 + 81x + 14
Cnt classes 7
True
Find char pol 
1x^13 + 101x^11 + 11x^10 + 62x^9 + 56x^8 + 1x^7 + 22x^6 + 18x^5 + 78x^4 + 102x^3 + 84x^2 + 69x + 73
Cnt classes 3
True
Find char pol 
1x^18 + 34x^16 + 14x^15 + 31x^14 + 22x^13 + 17x^12 + 20x^11 + 11x^10 + 4x^9 + 17x^7 + 9x^6 + 40x^5 + 17x^4 + 22x^3 + 22x^2 + 43x + 23
Cnt classes 3
True
Find char pol 
1x^13 + 37x^11 + 35x^10 + 17x^9 + 64x^8 + 78x^7 + 109x^6 + 90x^5 + 103x^

Далее напишем тест для факто-колец

In [74]:
def test_bm_lrs_factor_ring():
    R = IntegerModRing(3)
    Rx = PolynomialRing(R, gen_name='\u03b2')
    pol = Rx([R(1), R(1), R(1)])  # x^2 + x + 1
    I = PrincipalIdealOverPolynomialRing(Rx, [pol], is_left=True, name='I')
    RxI = FactorRingOverPrincipalPolynomialIdeal(Rx, I)  # GF(4)

    beta = RxI([R(0), R(1)])  # [x]
    e_RxI = RxI.identity()
    RxIy = PolynomialRing(RxI, gen_name='y')
    poly = RxIy([beta, beta, e_RxI])
    J = PrincipalIdealOverPolynomialRing(RxIy, [poly], is_left=True, name='J')
    RxIyJ = FactorRingOverPrincipalPolynomialIdeal(RxIy, J)  # GF(16) if R = GF(2)
    RxIyJz = PolynomialRing(RxIyJ, 'x')
    x = RxIyJz.gen()

    coefs_indicies = np.random.randint(0, RxIyJ.cardinality(), 4)
    coefs = [RxIyJ.idx_to_elem(c) for c in coefs_indicies]
    char_pol = x ** 5 - RxIyJz(coefs)

    u = LRSoverRing(RxIyJ, [RxIyJ.zero()] * 4 + [RxIyJ.identity()], char_pol)
    u.compute_n_terms(10)
    bm = BerlecampMasseyVLOverFCR(verbose=True)
    bm.fit(u)
    find_pol = bm.hist[0][-1]["g"]
    print(f'True cp {char_pol}')
    print(f'Found   {find_pol}')
    assert char_pol == find_pol
    print(char_pol == find_pol)
    
test_bm_lrs_factor_ring()

Split ring <__main__.FactorRingOverPrincipalPolynomialIdeal object at 0x7f891a25b790> to left assoc classes
Cnt classes 4
Step number 0
After step idx 0:
table 0
g_0: 1 + I + J
k_0: 4
m_0: 0
u_0: [0 + I + J, 0 + I + J, 0 + I + J, 0 + I + J, 1 + I + J, 0 + I + J, (1β + 2 + I)y + (2β + 2 + I) + J, (1 + I)y + (1β + 2 + I) + J, (2β + 2 + I)y + (2 + I) + J, (2 + I)y + (1β + 1 + I) + J, (2β + 2 + I)y + (2β + I) + J, (1β + 2 + I)y + (1β + 1 + I) + J, (1β + 1 + I)y + (1β + I) + J, (2 + I)y + (2β + I) + J, (1 + I)y + (1β + 2 + I) + J]

table 1
g_0: 1β + 2 + I + J
k_0: 4
m_0: 0
u_0: [0 + I + J, 0 + I + J, 0 + I + J, 0 + I + J, 1β + 2 + I + J, 0 + I + J, 1β + 2 + I + J, (1β + 2 + I)y + J, (1β + 2 + I)y + (2β + 1 + I) + J, (2β + 1 + I)y + (2β + 1 + I) + J, (1β + 2 + I)y + (2β + 1 + I) + J, 2β + 1 + I + J, (2β + 1 + I)y + (1β + 2 + I) + J, (2β + 1 + I)y + (2β + 1 + I) + J, (1β + 2 + I)y + J]

table 2
g_0: (1 + I)y + (2 + I) + J
k_0: 4
m_0: 0
u_0: [0 + I + J, 0 + I + J, 0 + I + J, 0 + I + J, (1 + I)

Задание. Напишите хорошие тесты для вашей реализации АБМ. В качестве верных ответов для некоторых тестов можете взять уже написаннай вами БМ над полем и над кольцом вычетов $\mathbb{Z}_n$, $n=p_1p_2\ldots p_k$

## 4. Аннулятор ЛРП

Используя Вашу реализацию. Напишите функцию для построения аннулятора последовательности и нахождения всех минимальных многочленов. Напишите хорошие тесты

# 5*. Задание для молодцов. 

Реализуйте алгоритм БМ для последовательности над кольцом матриц над полем. Ну и конечно же протестируйте

**ps** С уже написанными функциями это можно сделать в 20-30 строк кода